In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import keras
import os
import time
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
labels = os.listdir('../input/kermany2018/OCT2017 /train/')

train_datagen = ImageDataGenerator(samplewise_center=True, 
                              samplewise_std_normalization=True, 
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.1, 
                              width_shift_range=0.1, 
                              rotation_range=15, 
                              zoom_range=0.15)
validation_datagen = ImageDataGenerator(samplewise_center=True, 
                                        samplewise_std_normalization=True)
test_datagen = ImageDataGenerator(samplewise_center=True, 
                                  samplewise_std_normalization=True)

In [ ]:
train_data_dir = '../input/kermany2018/OCT2017 /train'
validation_data_dir = '../input/kermany2018/OCT2017 /val'
test_data_dir = '../input/kermany2018/OCT2017 /test'

In [ ]:
image_size = 224
valid_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(image_size,image_size),
    batch_size=32,
    class_mode='categorical')
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_size,image_size),
    batch_size=42,
    class_mode='categorical',
    shuffle=True)

In [ ]:
# defining simpleCNN model
t_x, t_y = next(train_generator)

def conv_layer():
    model=Sequential()
    model.add(Conv2D(64,(5,5),activation='relu',input_shape=(224,224,3),kernel_initializer='he_normal'))
    model.add(Conv2D(64,(5,5),activation='relu',padding='same',kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(32,(5,5),activation='relu',padding='same',kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(32,5,activation='relu',padding='same',kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(16,(5,5),activation='relu',padding='same',kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(64,activation='relu',kernel_initializer='he_normal'))
    model.add(Dense(32,activation='relu',kernel_initializer='he_normal'))
    model.add(Dense(4,activation='softmax'))
    return model

In [ ]:
# compiling model
model = conv_layer()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                           metrics = ['accuracy'])

In [ ]:
# creating checkpoint
file_path = "weights.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# training the model
history = model.fit_generator(train_generator, 
                                  steps_per_epoch=100,
                                  validation_data = valid_generator, 
                                  epochs = 20,
                                  callbacks=callbacks_list)

In [ ]:
# loading the model weights

model = conv_layer()
model.load_weights(file_path)

In [ ]:
# creating test data generator

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(image_size,image_size),
    batch_size=968,
    class_mode='categorical')
test_X,test_Y = next(test_generator)

In [ ]:
# predicting y value and measuring test time per image

start = time.time()
y_pred = model.predict(test_X)
end = time.time()

print('Test time per image:')
print((end - start)/968)

In [ ]:
Y_pred = []
for i in range(len(y_pred)):
    Y_pred.append(np.argmax(y_pred[i]))
    
Y_test = []
for i in range(len(test_Y)):
    Y_test.append(np.argmax(test_Y[i]))
    
print(classification_report(Y_test, Y_pred))

In [ ]:
# Creating a confusion matrix represented as a heat map
labels = [0,1,2,3]
sns.heatmap(confusion_matrix(Y_test, Y_pred), annot=True, fmt=".3f", xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')
plt.show()